# Selecting Continuous Life-Like Cellular Automata for Halting Unpredictability: Evolving for Abiogenesis.

In [ ]:
# this section needs to be run for setup on colab
# uncomment and run the commands below if on colab
# then restart the runtime and run tests in the next cell


#! git clone https://github.com/riveSunder/yuca.git my_yuca
#%cd my_yuca
#! pip install -e .

In [ ]:
#! python -m testing.test_all

In [ ]:
# make  cells wider
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from random import shuffle
import numpy as np

import torch

from yuca.multiverse import CA
from yuca.cmaes import CMAES
from yuca.zoo.librarian import Librarian
from yuca.halting_wrapper import HaltingWrapper, SimpleHaltingWrapper

import matplotlib.pyplot as plt
import matplotlib

my_cmap = plt.get_cmap("magma")
plt.rcParams['figure.dpi'] = 300

matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42
matplotlib.rcParams["animation.embed_limit"] = 256
matplotlib.rcParams["font.size"] = 10
#matplotlib.rcParams["text.usetex"] = True

import IPython


In [ ]:
# functions for plotting animations

def plot_4x4(grids):
    # grids is a length 16 list of tensors containing CA grids
    
    global subplot_0
    
    my_cmap = plt.get_cmap("magma")
    fig, ax = plt.subplots(1,1, figsize=(2.25, 2.25), facecolor="white")
    
    # combine grids
    # random CA rule parameters
    grid = torch.cat([grids[0], grids[1], grids[2], grids[3]], dim=-1)
    # lenia
    grid = torch.cat([grid, torch.cat([grids[4], grids[5], grids[6], grids[7]], dim=-1)], dim=-2)
    # evolved 1
    grid = torch.cat([grid, torch.cat([grids[8], grids[9], grids[10], grids[11]], dim=-1)], dim=-2)
    # evolved 2
    grid = torch.cat([grid, torch.cat([grids[12], grids[13], grids[14], grids[15]], dim=-1)], dim=-2)
    
    disp_grid =  1.0 - my_cmap(grid.squeeze())[...,0:3]
    
    subplot_0 = ax.imshow(disp_grid)
    
    ax.set_title("CA grids")
    tick_step = (grid.shape[-2]//4 )
    ax.set_yticks([tick_step//2 + tick_step*kk for kk in range(4)])
    ax.set_yticklabels(['typical random rules (a)', 'Lenia rules (b)', \
                        'evo. rules (simple) (c)', 'evo. rules (halting) (d)'])
    ax.set_xticklabels('')
    return fig, ax
    
def update(hh):
    
    with torch.no_grad():
        global grids
        global population
        global subplot_0

        my_cmap = plt.get_cmap("magma")

        for ii in range(len(grids)):
            grids[ii] = population[ii](grids[ii])

        # combine grids
        # random CA rule parameters
        grid = torch.cat([grids[0], grids[1], grids[2], grids[3]], dim=-1)
        # lenia
        grid = torch.cat([grid, torch.cat([grids[4], grids[5], grids[6], grids[7]], dim=-1)], dim=-2)
        # evolved 1
        grid = torch.cat([grid, torch.cat([grids[8], grids[9], grids[10], grids[11]], dim=-1)], dim=-2)
        # evolved 2
        grid = torch.cat([grid, torch.cat([grids[12], grids[13], grids[14], grids[15]], dim=-1)], dim=-2)

        disp_grid =  1.0 - my_cmap(grid.squeeze())[...,0:3]

        subplot_0.set_array(disp_grid)
        ax.set_yticklabels(['a','b','c','d'])
        ax.set_xticklabels('')
        

## Section 1: 
### Visualize typical CA dynamics: evolved with random fitness, complex Lenia CA, and CA with evolved rules


In [ ]:
ca = CA()
cmaes = CMAES(population_size=16)
use_random_evo = True

lib = Librarian()
lib.index
lenia_configs = ["orbium.npy", "p_sinus_labens.npy", "geminium.npy", "synorbium.npy"]
random_configs = ["exp_local_glaberish_orbium_random_1649112702_seed3_config.npy",\
                 "exp_local_glaberish_orbium_random_1649112702_seed5_config.npy",\
                 "exp_local_glaberish_orbium_random_1649112702_seed7_config.npy",\
                 "ca_configs/exp_local_glaberish_geminium_random_1649461964_seed113.npy"]
evo_configs = [\
               "s643.npy", \
               "exp_local_glaberish_geminium_simple_halt_1649264299_seed11_config.npy",\
               "exp_local_glaberish_geminium_simple_halt_1649264299_seed7_config.npy",\
               "exp_local_glaberish_geminium_simple_halt_1649312229_seed5_config.npy",\
               "exp_local_glaberish_orbium_halt_1649112713_seed3_config.npy",\
               "exp_local_glaberish_orbium_halt_1649112713_seed7_config.npy",\
               "s613.npy", \
               "exp_local_glaberish_geminium_halt_1651935163_seed5.npy"
              ]

#"exp_local_glaberish_orbium_halt_1649112713_seed3_config.npy", \

In [ ]:
cmaes.reset()
cmaes.initialize_population()

population = []


for jj in range(4):
    
    ca = CA()
    ca.no_grad()
    ca.set_params(cmaes.population[jj].get_action())
    
    if use_random_evo:
        ca.restore_config(random_configs[jj])
        
    population.append(ca)

for jj in range(4,8):
    # load lenia configs
    ca = CA()
    ca.no_grad()
    ca.restore_config(lenia_configs[jj-4])

    population.append(ca)

for jj in range(8,16):
    # load lenia configs
    ca = CA()
    ca.no_grad()
    ca.restore_config(evo_configs[jj-8])
    
    population.append(ca)

    

In [ ]:
#3141592
torch.manual_seed(314)
rand_grid = torch.rand(1,1,96,96)
rand_grid.requires_grad = False
grids = [1.0 * rand_grid] * 16

num_frames = 100
fig, ax = plot_4x4(grids)
#plt.close("all")
plt.show()

save_gif = False
if (save_gif):
    matplotlib.animation.FuncAnimation(fig, update, frames=num_frames, interval=100).save("uniform_init_cas.gif")
    grids = [1.0 * rand_grid] * 16
else:
    pass

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update, frames=num_frames, interval=100).to_jshtml())

## Section 2
### Try predicting CA halting for yourself. How hard is it to predict whether a complex CA with vanish? 

In [ ]:
# how easy/hard is it to predict halting? 
# try it yourself and see


torch.manual_seed(42)

my_configs = ["orbium.npy", \
              "s613.npy",\
             "exp_local_glaberish_orbium_random_1649112702_seed3_config.npy",\
             "exp_local_glaberish_geminium_simple_halt_1649264299_seed11_config.npy"]

shuffle(my_configs)
batch_size = 4
first_steps = 13
ca_steps = 256
ready_to_go = True
over_count = 0

for my_config in my_configs:
    
    if ready_to_go:
        pass
    else:
        break
    
    with torch.no_grad():
        ready = False
        while not ready:
            ca = CA()

            ca.restore_config(my_config)

            # 'training set'
            grids = torch.rand(batch_size,1,96,96)

            for step in range(first_steps):
                grids = ca(grids)

            first_grids = grids * 1.0

            for step in range(ca_steps):
                grids = ca(grids)

            fig, ax = plt.subplots(2, batch_size, figsize=( batch_size,2))

            for xx in range(batch_size):
                ax[0,xx].imshow(first_grids[xx].squeeze())
                ax[1,xx].imshow(grids[xx].squeeze())

                ax[0,xx].set_xticklabels('')
                ax[0,xx].set_yticklabels('')
                ax[1,xx].set_xticklabels('')
                ax[1,xx].set_yticklabels('')

                ax[0,xx].set_title(f"{xx}")
                    
            ax[0,0].set_yticks([grids.shape[-2]//2])
            ax[0,0].set_yticklabels([f"{first_steps} steps"], fontsize=3, rotation=0)

            ax[1,0].set_yticks([grids.shape[-2]//2])
            ax[1,0].set_yticklabels([f"{ca_steps} steps"], fontsize=3, rotation=0)
            plt.suptitle("training", y=1.1)
            #plt.tight_layout()

            
            ready = bool(int(input("ready to predict 1/0?")))
        
            plt.show()

        # 'training set'
        grids = torch.rand(batch_size,1,96,96)

        for step in range(first_steps):
            grids = ca(grids)

        first_grids = grids * 1.0
        
        for step in range(ca_steps):
            grids = ca(grids)

        fig, ax = plt.subplots(2, batch_size, figsize=(batch_size,2))
        
        for xx in range(batch_size):
            ax[0,xx].imshow(first_grids[xx].squeeze())
                      
            ax[0,xx].set_xticklabels('')
            ax[0,xx].set_yticklabels('')
            ax[1,xx].set_xticklabels('')
            ax[1,xx].set_yticklabels('')
            
            ax[0,xx].set_title(f"{xx}")
  
        ax[0,0].set_yticks([grids.shape[-2]//2])
        ax[0,0].set_yticklabels([f"{first_steps} steps"], fontsize=3, rotation=0)
        
        ax[1,0].set_yticks([grids.shape[-2]//2])
        ax[1,0].set_yticklabels([f"{ca_steps} steps"], fontsize=3, rotation=0)
            
        plt.suptitle("inference", y=1.1)
        #plt.tight_layout()

        plt.show()
        
        guesses = []
        
        for ll in range(4):
            
            guesses.append(bool(int(input(f"Will grid {ll} vanish?"))))
            

        for step in range(ca_steps):
            grids = ca(grids)

        fig, ax = plt.subplots(2, batch_size, figsize=(batch_size,2))

        
        actual = []
        for ll in range(4):
            actual.append(grids[ll].mean().item() == 0.0)
            
        for xx in range(batch_size):
            ax[0,xx].imshow(first_grids[xx].squeeze())
            ax[1,xx].imshow(grids[xx].squeeze())

            ax[0,xx].set_xticklabels('')
            ax[0,xx].set_yticklabels('')
            ax[1,xx].set_xticklabels('')
            ax[1,xx].set_yticklabels('')
            
            ax[0,xx].set_title(f"{guesses[xx] == actual[xx]}") 
        

        ax[0,0].set_yticks([grids.shape[-2]//2])
        ax[0,0].set_yticklabels([f"{first_steps} steps"], fontsize=3, rotation=0)
        
        ax[1,0].set_yticks([grids.shape[-2]//2])
        ax[1,0].set_yticklabels([f"{ca_steps} steps"], fontsize=3, rotation=0)
        plt.suptitle("inference check", y=1.1)
        #plt.tight_layout()

        plt.show()
        
        actual = []
        for ll in range(4):
            actual.append(grids[ll].mean().item() == 0.0)
        
        correct_count = np.sum(np.array(actual) == np.array(guesses))
        
        msg = f"\n\n *****accuracy for config {my_config} is {correct_count} of {len(actual)}******\n\n"
        
        print(msg)
        
        over_count += 1
        
        if over_count == len(my_configs):
            break
        ready_to_go = bool(int(input("\nwant to make some more halting predictions?\n")))


## Section 3
### Evolve CA rules for halting unpredictability, or for roughly even proportion of vanishing and persistent cell values

* HaltingWrapper: generates a fitness value based on training convolutional neural networs to predict vanishing or persistent patterns
* SimpleHaltingWrapper: selects based on an even proportion of vanishing and persistent patterns (no prediction). 

In [ ]:
# evolve your own CA 
my_device = "cuda" if torch.cuda.is_available() else "cpu"
visualize = True
torch.manual_seed(42)
np.random.seed(42)

# SimpleHaltingWrapper selects for an even proportion of vanishing and persistent grids
# change SimpleHaltingWrapepr to HaltingWrapper for halting unpredictability evollution
kwargs = {"env_fn": SimpleHaltingWrapper,\
          "batch_size": 64,\
          "tag": "orbish",\
          "population_size": 16,\
          "generations": 10,\
          "replicates": 1,\
          "device": my_device,\
          "ca_steps": 256,\
          "kernel_radius": 13,\
          "dim": 128\
         }

cmaes = CMAES(**kwargs)
cmaes.reset()
cmaes.initialize_population()

In [ ]:
if visualize:
    population = []


    for jj in range(cmaes.population_size):

        ca = CA(tag=kwargs["tag"])
        ca.no_grad()
        ca.set_params(cmaes.population[jj].get_action())

        population.append(ca)
        
    torch.manual_seed(31415)
    rand_grid = torch.rand(1,1,96,96)
    rand_grid.requires_grad = False
    grids = [1.0 * rand_grid] * 16

    # changed to 75 due to problems with higher num_frames in colab
    num_frames = 75 #kwargs["ca_steps"]
    fig, ax = plot_4x4(grids)
    #plt.close("all")
    plt.show()

else:
    num_frames = 0
    
    rand_grid = torch.rand(1,1,96,96)
    grids = [1.0 * rand_grid] * 16
    fig, ax = plot_4x4(grids)

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update, frames=num_frames, interval=100).to_jshtml())

In [ ]:
# run evolution 
cmaes.search()

In [ ]:
if visualize:
    population = []


    for jj in range(cmaes.population_size):

        ca = CA(tag=kwargs["tag"])
        ca.no_grad()
        ca.set_params(cmaes.population[jj].get_action())

        population.append(ca)
        
    torch.manual_seed(31415)
    rand_grid = torch.rand(1,1,96,96)
    rand_grid.requires_grad = False
    grids = [1.0 * rand_grid] * 16

    # changed to 75 due to problems with higher num_frames in colab
    num_frames = 75 #kwargs["ca_steps"]
    fig, ax = plot_4x4(grids)
    #plt.close("all")
    plt.show()

else:
    num_frames = 0
    
    rand_grid = torch.rand(1,1,96,96)
    grids = [1.0 * rand_grid] * 16
    fig, ax = plot_4x4(grids)

save_gif = False
if (save_gif):
    matplotlib.animation.FuncAnimation(fig, update, frames=num_frames, interval=100).save("evolved_geminish_ca.gif")
    grids = [1.0 * rand_grid] * 16
else:
    pass

    
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update, frames=num_frames, interval=100).to_jshtml())